Importing necessary libraries

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

Download data for various indices. Due to some problem in yfinance, it was not able to download all of the specified indices so more than 5 indices are supplied.

In [16]:
tickers = ['^GSPC', '^DJI', '^FTSE', '^GDAXI', '^FCHI', '^KS11', '^IXIC', '^N225']
data = yf.download(tickers, start='2010-01-01', end='2023-05-01')

[*********************100%***********************]  8 of 8 completed

3 Failed downloads:
- ^IXIC: No data found for this date range, symbol may be delisted
- ^GDAXI: No data found for this date range, symbol may be delisted
- ^N225: No timezone found, symbol may be delisted


Download data for some equities

In [75]:
equity_tickers = ['GOOG', 'ADANIENT.NS', 'MSFT', 'ONGC.BO', 'RELIANCE.NS']
equity_data = yf.download(equity_tickers, start='2010-01-01', end='2023-05-01')

[*********************100%***********************]  5 of 5 completed


Organise the downloaded data into multiple dataframes, each containing the data related to itself

In [55]:
indices_list = []
for ticker in tickers:
    df = pd.DataFrame()
    for col in ['Adj Close', 'Close', 'Open', 'High', 'Low', 'Volume']:
        df[col] = data[col][ticker]

    if pd.isna(df['Adj Close'][-1]):
        continue
    indices_list.append((ticker, df))


In [83]:

equities_list = []
for ticker in equity_tickers:
    df = pd.DataFrame()
    for col in ['Adj Close', 'Close', 'Open', 'High', 'Low', 'Volume']:
        df[col] = equity_data[col][ticker]

    if pd.isna(df['Adj Close'][-1]):
        continue
    equities_list.append((ticker, df))


Define the functions to calculate the desired quantities.

In [4]:
def daily_returns(df):
    return pd.DataFrame((df['Close']-df['Open']) / df['Open'])

def volatility(df):
    return np.float64(daily_returns(df).std() * np.sqrt(252))

def cumulative_returns(df):
    return np.float64((1+daily_returns(df)).product(axis=0) - 1)

def sharpe_ratio(df, risk_free_rate, N):
    return np.float64((daily_returns(df).mean()*N - risk_free_rate) / volatility(df))

def sortino_ratio(df, target_returns, N):
    returns = daily_returns(df)
    return np.float64((returns.mean() * N - target_returns) / (returns[returns < 0].std() * np.sqrt(N)))

def max_drawdown(df):
    return np.float64((df['Low'].min() - df['High'].max()) / df['High'].max() * 100)


Write the obtained data to a file, to generate the README markdown file.

In [74]:
with open('international_indices.md', 'w') as readme:
    readme.write('Data | ')
    for index in indices_list:
        readme.write(index[0] + '| ')
    readme.write('\n')
    readme.write("----|----|----|----|----|----\n")
    readme.write('Cumulative Returns | ')
    for index in indices_list:
        readme.write(str(cumulative_returns(index[1]).round(3)) + '| ')
    readme.write('\n')
    readme.write('Volatility | ')
    for index in indices_list:
        readme.write(str(volatility(index[1]).round(3)) + '| ')
    readme.write('\n')
    readme.write('Sharpe Ratio | ')
    for index in indices_list:
        readme.write(str(sharpe_ratio(index[1], risk_free_rate=3.5/100, N=252).round(3)) + '| ')
    readme.write('\n')
    readme.write('Sortino Ratio | ')
    for index in indices_list:
        readme.write(str(sortino_ratio(index[1], target_returns=3.5/100, N=252).round(3)) + ' | ')
    readme.write('\n')
    readme.write('Max Drawdown | ')
    for index in indices_list:
        readme.write(str(max_drawdown(index[1]).round(3)) + '| ')

In [ ]:
with open('international_indices.md', 'w') as readme:
    readme.write('Data | ')
    for index in equities_list:
        readme.write(index[0] + '| ')
    readme.write('\n')
    readme.write("----|----|----|----|----|----\n")
    readme.write('Cumulative Returns | ')
    for index in equities_list:
        readme.write(str(cumulative_returns(index[1]).round(3)) + '| ')
    readme.write('\n')
    readme.write('Volatility | ')
    for index in equities_list:
        readme.write(str(volatility(index[1]).round(3)) + '| ')
    readme.write('\n')
    readme.write('Sharpe Ratio | ')
    for index in equities_list:
        readme.write(str(sharpe_ratio(index[1], risk_free_rate=3.5/100, N=252).round(3)) + '| ')
    readme.write('\n')
    readme.write('Sortino Ratio | ')
    for index in equities_list:
        readme.write(str(sortino_ratio(index[1], target_returns=3.5/100, N=252).round(3)) + ' | ')
    readme.write('\n')
    readme.write('Max Drawdown | ')
    for index in equities_list:
        readme.write(str(max_drawdown(index[1]).round(3)) + '| ')